## Загрузка и первоначальный осмотр данных

In [2]:
import pandas as pd
import numpy as np

df_titanic = pd.read_csv('Titanic.csv')

## Задание 1: Предобработка данных

In [8]:
initial_rows = len(df_titanic)
print(f"Начальное количество строк: {initial_rows}")

df_titanic_cleaned = df_titanic.dropna()
rows_after_dropna = len(df_titanic_cleaned)
print(f"Количество строк после удаления пропусков (dropna): {rows_after_dropna}")

df_before_column_drop = df_titanic_cleaned.copy()

non_numeric_cols = df_titanic_cleaned.select_dtypes(include=['object']).columns
cols_to_drop = [col for col in non_numeric_cols if col not in ['Sex', 'Embarked']]
df_titanic_cleaned = df_titanic_cleaned.drop(columns=cols_to_drop)
print(f"Столбцы после удаления нечисловых (кроме Sex, Embarked): {df_titanic_cleaned.columns.tolist()}")

df_titanic_cleaned['Sex'] = df_titanic_cleaned['Sex'].map({'male': 0, 'female': 1})
df_titanic_cleaned['Embarked'] = df_titanic_cleaned['Embarked'].map({'C': 1, 'Q': 2, 'S': 3})

if 'PassengerId' in df_titanic_cleaned.columns:
    df_titanic_cleaned = df_titanic_cleaned.drop(columns=['PassengerId'])
    print("Столбец 'PassengerId' удален.")
else:
    print("Столбец 'PassengerId' уже был удален или отсутствовал.")

loss_after_step1_1 = (initial_rows - rows_after_dropna) / initial_rows * 100
print(f"\nПроцент потерянных данных: {loss_after_step1_1:.2f}%")

Начальное количество строк: 891
Количество строк после удаления пропусков (dropna): 183
Столбцы после удаления нечисловых (кроме Sex, Embarked): ['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
Столбец 'PassengerId' удален.

Процент потерянных данных: 79.46%


## Задание 2: Машинное обучение

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


X = df_titanic_cleaned.drop('Survived', axis=1)
y = df_titanic_cleaned['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Размер обучающей выборки: {X_train.shape}, тестовой: {X_test.shape}")


model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nТочность модели логистической регрессии (со всеми признаками): {accuracy:.4f}")

print("\nОценка влияния признака 'Embarked':")
X_no_embarked = X.drop('Embarked', axis=1)
X_train_ne, X_test_ne, y_train_ne, y_test_ne = train_test_split(X_no_embarked, y, test_size=0.3, random_state=42, stratify=y)

model_ne = LogisticRegression(random_state=42, max_iter=1000)
model_ne.fit(X_train_ne, y_train_ne)
y_pred_ne = model_ne.predict(X_test_ne)
accuracy_ne = accuracy_score(y_test_ne, y_pred_ne)
print(f"Точность модели без признака 'Embarked': {accuracy_ne:.4f}")
print(f"Разница в точности: {accuracy - accuracy_ne:.4f}")


Размер обучающей выборки: (146, 7), тестовой: (37, 7)

Точность модели логистической регрессии (со всеми признаками): 0.7838

Оценка влияния признака 'Embarked':
Точность модели без признака 'Embarked': 0.7455
Разница в точности: 0.0383
